In [1]:
# Instalar la biblioteca faltante
!pip install resampy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 33.5 MB/s eta 0:00:00


In [2]:
# Importar las bibliotecas necesarias
import numpy as np
import os
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [3]:
# Descargar y preparar los datos
!wget -q https://url_del_dataset.zip
!unzip -q dataset.zip

In [4]:
# Definir las clases y los directorios de los datos
classes = ['uno', 'dos', 'tres', 'cuatro', 'cinco', 'fondo']
data_dir = 'dataset'

In [5]:
# Función para extraer características de los audios
def extract_features(file_name):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    return mfccs

In [6]:
# Cargar los datos de los audios y sus etiquetas
data = []
labels = []
for i, _class in enumerate(classes):
    dir_path = os.path.join(data_dir, _class)
    for filename in os.listdir(dir_path):
        file_path = os.path.join(dir_path, filename)
        mfccs = extract_features(file_path)
        data.append(mfccs.T) # Transponer las características
        labels.append(i)

In [7]:
# Convertir los datos y las etiquetas a matrices numpy
data = np.asarray(data)
labels = np.asarray(labels)

<ipython-input-7-df279bdf08f5>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.asarray(data)


In [8]:
# Ajustar todas las características a la misma longitud mediante relleno o truncamiento
max_length = max(len(seq) for seq in data)
data = tf.keras.preprocessing.sequence.pad_sequences(data, maxlen=max_length)

In [9]:
# Dividir los datos en conjuntos de entrenamiento y prueba
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

In [10]:
# Preprocesar los datos
train_data = tf.keras.utils.normalize(train_data, axis=2)  # Normalizar en el eje de características
test_data = tf.keras.utils.normalize(test_data, axis=2)  # Normalizar en el eje de características

In [11]:
# Construir el modelo de clasificación de audio
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=train_data.shape[1:]))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(len(classes), activation=tf.nn.softmax))

In [12]:
# Compilar y entrenar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, train_labels, epochs=100, batch_size=32, validation_data=(test_data, test_labels))

Epoch 1/100
9/9 [==============================] - 1s 23ms/step - loss: 1.6438 - accuracy: 0.5477 - val_loss: 1.4447 - val_accuracy: 0.8028
Epoch 2/100
9/9 [==============================] - 0s 5ms/step - loss: 1.1932 - accuracy: 0.9753 - val_loss: 0.9894 - val_accuracy: 0.9718
Epoch 3/100
9/9 [==============================] - 0s 8ms/step - loss: 0.6995 - accuracy: 0.9859 - val_loss: 0.5462 - val_accuracy: 0.9577
Epoch 4/100
9/9 [==============================] - 0s 5ms/step - loss: 0.3249 - accuracy: 0.9965 - val_loss: 0.2873 - val_accuracy: 0.9718
Epoch 5/100
9/9 [==============================] - 0s 8ms/step - loss: 0.1406 - accuracy: 0.9965 - val_loss: 0.1663 - val_accuracy: 0.9718
Epoch 6/100
9/9 [==============================] - 0s 5ms/step - loss: 0.0665 - accuracy: 0.9965 - val_loss: 0.1144 - val_accuracy: 0.9718
Epoch 7/100
9/9 [==============================] - 0s 5ms/step - loss: 0.0406 - accuracy: 0.9965 - val_loss: 0.0951 - val_accuracy: 0.9718
Epoch 8/100
9/9 [=========

In [13]:
# Evaluar el modelo con los datos de prueba
loss, accuracy = model.evaluate(test_data, test_labels)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

3/3 [==============================] - 0s 3ms/step - loss: 0.0250 - accuracy: 0.9859
Loss: 0.02499721571803093
Accuracy: 0.98591548204422


In [14]:
# Guardar el modelo entrenado
model.save('audio_classification_model.h5')

In [15]:
# Convertir el modelo a formato TensorFlow.js
!pip install tensorflowjs
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, 'audio_classification_model_js')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.9/189.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 23.1
    Uninstalling packaging-23.1:
      Successfully uninstalled packaging-23.1
  Attempting uninstall: flax
    Found existing installation: flax 0.6.9
    Uninstalling flax-0.6.9:
      Successfully uninstalled flax-0.6.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
statsmodels 0.13.5 requires packaging>=21.3, but you have packaging 20.9 which is incompatible.
xar

In [16]:
# Leer el modelo convertido en formato JavaScript
with open('audio_classification_model_js/model.json', 'r') as file:
    model_js_code = file.read()

In [17]:
# Guardar el modelo como un archivo de origen C Header (.h)
with open('audio_classification_model.h', 'w') as file:
    file.write('#ifndef AUDIO_CLASSIFICATION_MODEL_H\n')
    file.write('#define AUDIO_CLASSIFICATION_MODEL_H\n\n')
    file.write('#include <stddef.h>\n\n')
    file.write('extern const unsigned char audio_classification_model[];\n')
    file.write('extern const size_t audio_classification_model_len;\n\n')
    file.write('const unsigned char audio_classification_model[] = {\n')
    # Escribir el código del modelo convertido en formato JavaScript
    file.write(model_js_code)

    file.write('};\n')
    file.write('const size_t audio_classification_model_len = sizeof(audio_classification_model);\n\n')
    file.write('#endif\n')